### Lawson-Hanson NNLS Algoritam - Metoda aktivnih skupova

Metoda aktivnih skupova je efikasna tehnika za rešavanje problema optimizacije sa ograničenjima. 
Naročito je korisna u problemima koji se mogu modelovati kao minimizacija funkcije uz nejednakosna ograničenja. 
Koristi princip aktivnih skupova kako bi pronašla optimalna rešenja, fokusirajući se na skup ograničenja koja su "aktivna" ili su na granici dozvoljenog prostora rešenja.

#### Ključni Koncepti:
* Aktivni Skup ($P$): Ovo je skup indeksa promenljivih koje su trenutno deo rešenja. Aktivni skup se stalno menja tokom iteracija kako bi se uključili novi atributi koji mogu poboljšati rešenje.
* Neaktivni Skup ($R$): Ovo je skup indeksa promenljivih koje trenutno nisu deo rešenja. Promenljive iz ovog skupa se razmatraju za moguće uključivanje u aktivni skup u sledećim iteracijama.
* Gradijent ($w$): Ovaj vektor meri koliko svaka promenljiva doprinosi smanjenju greške ako bi bila uključena u rešenje. Gradijentni vektor pomaže u određivanju koje promenljive treba dodati u aktivni skup.

#### Opis Metode:
##### Inicijalizacija:
* Postavljamo inicijalne vrednosti za aktivni skup $P$, neaktivni skup $R$, i inicijalno rešenje $x$.
* Izračunavamo početni gradijentni vektor $w$, koji pokazuje koliko svaka promenljiva doprinosi grešci.
##### Dodavanje Varijabli u Aktivni Skup:
* Izaberemo promenljivu sa najvećim doprinosom iz neaktivnog skupa $R$ i dodamo je u aktivni skup $P$.
* Ova promenljiva se uklonja iz $R$ i dodaje u $P$.
##### Rešavanje Problema Metodom Najmanjih Kvadrata:
* Koristimo samo promenljive iz aktivnog skupa $P$ za rešavanje problema metodom najmanjih kvadrata. Ovo daje privremeno rešenje $s$.
##### Provera i Korekcija:
* Proveravamo da li je rešenje nenegativno. Ako nije, korigujemo rešenje tako da se zadovolji uslov pozitivnosti.
* Korekcija se vrši pomoću koraka $\alpha$ koji prilagođava rešenje tako da ne narušava uslove pozitivnosti.
##### Ažuriranje i Iteracija:
* Ako su sve vrednosti pozitivne, ažuriramo konačno rešenje i gradijentni vektor.
* Proces se ponavlja dok ne dođe do konvergencije, tj. dok ne prestane da se menja najveći doprinos u gradijentnom vektoru ili dok ne bude zadovoljena tolerancija $\epsilon$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [15]:
def active_set_method(A, b, epsilon):
    # Inicijalizacija neophodnih promenljvih
    m, n = A.shape
    P = []                   # Aktivni skup - na kraju ce sadrzati indekse atributa sadrzanih u resenju
    R = list(range(n))       # Neaktivni skup - sadrzi indekse atributa koje jos uvek nisu u resenju
    x = np.zeros(n)          # Aproksimacija resenja - inicajlno nula - iterativno se izgradjuje
    w = A.T @ (b - A @ x)    # Gradijent - meri koliko svaki atribut doprinosi smanjenju greske kada bi bio ukljucen u resenje
    
    # Sve dok R nije prazan i najveci doprinos je veci od tolerancije 
    while R and np.max(w[R]) > epsilon:
        # Izaberi atribut sa najvecim doprinosom i izbaci ga iz R i dodaj ga u P
        j = R[np.argmax(w[R])] 
        P.append(j)
        R.remove(j)

        # Resi problem metodom najmanjih kadvrata za matricu restrikovanu na atribute iz aktivnog skupa P
        AP = A[:, P]                                   # Restrikovano A
        s = np.zeros(n)                                # Tekuca aproksimacija - inicijalno nula
        s_P = np.linalg.pinv(AP.T @ AP) @ (AP.T @ b)   # Racunamo resenje problema
        s[P] = s_P                                     # Upisujemo resenje u konacnu aproksimaciju

        # Provera da li je resenje validno tj da li su sve koordinate vece od 0
        while np.min(s[P]) <= 0:
            # Ako nisu onda se aproksimacija koriguje

            # Racunamo korak alpha koji ce aproksimaciju pribliziti s tako da ne narusava uslov pozitivnosti
            alpha = np.min(x[P] / (x[P] - s[P]))
            x = x + alpha * (s - x)

            # Atributi koji nisu postali nula idu u P 
            P = [i for i in P if x[i] > 0]
            # Oni koji jesu idu u R
            R = [i for i in range(n) if i not in P]

            # Azuriranja za sledecu iteraciju
            AP = A[:, P]
            s_P = np.linalg.pinv(AP.T @ AP) @ (AP.T @ b)
            s[P] = s_P

        # Ako jesu azurira se konacno resenje
        x = s  
        print(x)

        # Azuriramo gradijentni vektor
        w = A.T @ (b - A @ x)

    return x

In [13]:
A = np.array([
 [1.         ,0.        ],
 [1.         ,0.11111111],
 [1.         ,0.22222222],
 [1.         ,0.33333333],
 [1.         ,0.44444444],
 [1.         ,0.55555556],
 [1.         ,0.66666667],
 [1.         ,0.77777778],
 [1.         ,0.88888889],
 [1.         ,1.        ]])

In [14]:
b = [50.024464,   49.88456337, 50.30240876, 50.29985166, 50.54841558, 50.53523403,
 50.65963392, 50.81760646, 50.83127725, 51.11296502]

In [16]:
active_set_method(A, b, 0.001)

[50.50164201  0.        ]
[49.95439673  1.09449054]


array([49.95439673,  1.09449054])